# 26. Ch 05. Advanced PyTorch Tutorials - 03. PyTorch Ignite

In [ ]:
PyTorch Ignite 라이브러리 활용해 딥러닝 학습을 효율적으로 만들기 위한 Building a Boilerplate
<보일러플레이트?>
일종의 템플릿이다
-학습 scheme이 비슷할 경우, 모델과 dataset을 제외한 코드는 거의 동일함
정작 중요한 모델을 코딩하는 시간보다 부수적인 요소 코딩에 더 많은 시간 소요
-따라서 모델과 dataset만 갈아끼워 재사용 가능한 코드가 있으면 좋을 것
-템플릿(Template)을 만들자

이그나이트는 보일러플레이트를 만들 수 있는 환경을 제공해준다

<Typical Model Training Prodcedure>
Epoch 시작 ->
-Training
Iteration 시작 -> Feed-forward -> Loss 계산 -> Back propagation -> 그레디언트 디센트 수행 -> 현재 상태 출력 -> Iteration 종료
-Validation
Iteration 시작 -> Feed-forward -> Loss 계산 -> 현재 상태 출력 -> Iteration 종료
->
Best loss 여부 체크 -> 모델 저장
-> Epoch 종료

파이토치는 전 과정을 직접 짜야 한다. 자유도가 높은 대신에 사용자의 노력이 필요.

<Interaction between Trainer and Others>
Train set, Valid set, Test set
Trainer: Training -> Validation -> Epoch 종료 -> Epoch 시작 -> 

# Validation set을 사용하는 이유 : 트레이닝 후에 만들어진 모델의 성능을 평가하기 위해.
# 트레이닝된 여러가지 모델 중 가장 좋은 모델을 고르기 위한 셋이다.
# 테스트 셋과는 달리 트레이닝 과정에 관여한다.
# 검증 데이터 셋을 활용해 모델을 튜닝해 모델의 성능을 높일 수 있다.

<Pytorch Ignite>
https://github.com/pytorch/ignite
-Pros:
파이토치의 불필요한 비효율성 해결
	-파이토치 공식 라이브러리
	(예. lightening은 공식 라이브러리가 아님)
	-Call-back 함수를 위한 이벤트가 잘 정의돼 있다
	-손 쉬운 학습 통계 logging(loss, accuracy…)
-Cons:
	-진입 장벽 존재
	-기본 설계의 아쉬움

# 콜백 함수: 전달된 함수를 다른 함수의 내부에서 호출

<Events for Call-back>
학습 Epoch 시작 ->
-Training
Iteration 시작 -> Feed-forward -> Loss 계산 -> Back propagation -> 그레디언트 디센트 수행 -> 현재 상태 출력 -> Iteration 종료
-> 학습 Epoch 종료 -> 검증 Epoch 시작 ->
-Validation
Iteration 시작 -> Feed-forward -> Loss 계산 -> 현재 상태 출력 -> Iteration 종료
->
Best loss 여부 체크 -> 모델 저장
-> 검증 Epoch 종료 -> 학습 Epoch 시작 ->

이벤트: Iteration 시작, Iteration 종료, 학습 Epoch 시작, 학습 Epoch 종료, 검증 Epoch 시작, 검증 Epoch 종료

<Engine 클래스>
CLASS ignite.engine.Engine(process_function>
for:
	def process_function: # 사용자가 직접 코딩
		Feed-forward
		Loss 계산
		Back-propagation
		Gradient Descent 수행
->
train_engine.run(
	train_loader,
	max_epochs=self.config.n_epochs,
)

<Attach Metrics>
-process_function의 리턴 값(예. 튜플)을 활용
def train(engine, mini_batch>:
	return float(loss), accuracy, p_norm, g_norm

pbar = ProgressBar(bar_format=None, ncols=120)
pbar.attach(train_engine, [‘|param|’, ‘|g_param|’, ‘loss’, ‘accuracy’])

<Add Event Handler (Call-back function)>
-add_event_handler 함수 활용
call-back 함수의 인터페이스가 복잡할 떄
def run_validation(engine, validation_engine, valid_loader):
	validation_engine.run(valid_loader, max_epochs=1)

train_engine.add_event_handler(
	Events.EPOCH_COMPLETED,
	run_validation,
	validation_engine,
	valid_loader,
)

-decorator 활용
콜백 함수의 인터페이스가 표준을 따를 때
@train_engine.on(Events.EPOCH_COMPLETED)
def print_train_logs(engine):
	avg_p_norm = engine.state.metrics[‘|param|’]
	avg_g_norm = engine.state.metrics[‘|g_param|’]
	avg_loss = engine.state.metrics[‘loss’]
	avg_accuracy = engine.state.metrics[‘accuracy]

	print(‘Epoch {} - |param|={:.2e} |g_param| = {:.2e} loss
		engine.state.epoch,
		avg_p_norm,
		avg_g_norm,
		avg_loss,
		avg_accuracy if avg_accuracy is not None else 0,
	))
이처럼 편리하게 이벤트를 등록해서
내가 원하는 시점에 코드들을 동작시킬 수 있다.

<Basic Strategy>
DataSet : __init__, __len__, __getitem__ ->
DataLoader : train, valid, test set으로 split ->
Run Train Engine : process_function, EPOCH_COMPLETED Print Statistics, EPOCH_COMPLETED Run Validation Engine
EPOCH_COMPLETED Run Validation Engine : process_function, EPOCH_COMPLETED Print Statistics
                           , EPOCH_COMPLETED Check Best Loss, EPOCH_COMPLETED Save Best Model

## 27. Ch 05. Advanced PyTorch Tutorials - 04. 실습 PyTorch Ignite를 활용하여 구현하기
-PyTorch Ignite 폴더 참고